In [ ]:
import numpy as np
import pandas as pd
import re
import cobra
import cobra.test
from cobra import Reaction, Metabolite
from cobra.flux_analysis import gapfill, production_envelope, flux_variability_analysis
from cobra.flux_analysis.gapfilling import GapFiller
from icecream import ic

In [ ]:
model = cobra.test.create_test_model("ecoli")
model_e = model.copy()

In [ ]:
model

for reaction in model_text.reactions[:5]:
    with model_text as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

In [ ]:
sol = model.optimize()

In [ ]:
print sol.fluxes

In [ ]:
model.summary()

In [ ]:
import requests
resp = requests.get('http://bigg.ucsd.edu/api/v2/models/iJO1366/genes')
res = resp.json()['results']
res[:5]
for gene in res:
    if gene['name'] == 'atoB':
        print gene
        break

In [ ]:
alc_dehydr = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='c')
model.add_reaction(alc_dehydr)
alc_dehydr.add_metabolites({'btcoa_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, 'btal_c': 1})

butanol = cobra.Metabolite(id='btol_c', name='1-butanol', compartment='c', charge=0, formula='C4H9OH')
but_synth = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='c')
model.add_reaction(but_synth)
but_synth.add_metabolites({'btal_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, butanol: 1})

In [ ]:
# TODO look at reversiblilty
# TODO look at whether we need to put custom enzymes in
# STEP 0 2 accoa_c -> aacoa_c + coa_c via atoB
model.reactions.ACACT1r
# STEP 1 aacoa_c + h_c + nadh_c -> nad_c + 3hbcoa_c via Hbd1
model.reactions.HACD1
# STEP 2 3hbcoa_c => b2coa_c + h2o_c via Crt
model.reactions.ECOAH1
# TODO: Change from fad_c to nad_c 
# STEP 3 b2coa_c + fadh2_c <=> btcoa_c + fad_c via Ter
model.reactions.ACOAD1f
# STEP 4 btcoa_c + h_c + nadh_c => nad_c + btal_c via adhE (b1241)
model.reactions.ALCDBUT
# STEP 5 btal_c + h_c + nadh_c => nad_c + btol_c via adhE
model.reactions.BUTSYN

In [ ]:
def opt(model):
    with model:
        if not ('DM_btol_c' in model.reactions):
            model.add_boundary(model.metabolites.btol_c, type='demand')
        model.objective = model.reactions.DM_btol_c
        obj = model.optimize()
        print obj.objective_value
        print model.summary()

In [ ]:
opt(model)

In [ ]:
def convert_cmpts(model, metab, from_cmpt='c', to_cmpt='e'):
    if type(metab) is cobra.Metabolite:
        metab = metab.id.rsplit('_', 1)[0]
    metab_name = metab + '_{0}'.format(to_cmpt)
    try:
        metab_cnvrt = model.metabolites.get_by_id(metab_name)
    except KeyError:
        metab_cnvrt = model.metabolites.get_by_id(metab + '_{0}'.format(from_cmpt)).copy()
        metab_cnvrt.id, metab_cnvrt.compartment = metab_name, to_cmpt
    return metab_cnvrt

In [ ]:
model.reactions.query(lambda x: med_met in x.metabolites and x.boundary)

In [ ]:
def remove_exs(model):
    model_no_ex = model.copy()
    model_no_ex.remove_reactions(reactions=model_no_ex.exchanges)
    return model_no_ex
model_no_ex = remove_exs(model)

In [ ]:
def add_ex(model, metab_dict, additive=False):
    rxn_dict = {}
    for med_met, conc in metab_dict.items():
        if conc < 0:
            ty = 'exchange'
            ty_name = 'EX'
        else:
            ty = 'demand'
            ty_name = 'DM'
        rxn_nm = '{0}_'.format(ty_name) + med_met.id
        if rxn_nm in model.reactions:
            rxn = model.reactions.get_by_id(rxn_nm)
            if additive:
                rxn.lower_bound = conc + rxn.lower_bound
            #    conc += rxn.upper_bound
            #    model.reactions.remove(rxn)
            #    model.add_boundary(metabolite=med_met, type='exchange', lb=conc)
            else:
                rxn.lower_bound = conc
        else:
            model.add_boundary(metabolite=med_met, type='medium-diff', reaction_id=rxn_nm, lb=conc, ub=1000)
        rxn_dict[model.reactions.get_by_id(rxn_nm)] = conc
        #print model.reactions.get_by_id(rxn_nm).upper_bound
    return rxn_dict

In [ ]:
def gen_metab_dict(model, metab_nms, metab_vals=None, cnvt=True, from_cmpt='c', to_cmpt='e'):
    metabs = []
    for metab_nm in metab_nms:
        if cnvt:
            metab = convert_cmpts(model, metab_nm, from_cmpt, to_cmpt)
        else:
            metab = metab_nm
        metabs.append(metab)
    if not metab_vals:
        metab_vals = [0] * len(metabs)
    metab_conc = dict(zip(metabs, metab_vals))
    return metab_conc

In [ ]:
def ecoli_to_cf(mod):
    # Delete transmembrane transport reactions
    """model = mod.copy()
    to_delete = []
    for rxn in model.reactions:
        if ('c' in rxn.compartments and 'p' in rxn.compartments) or \
            ('e' in rxn.compartments and 'p' in rxn.compartments):
            to_delete.append(rxn)
    model.remove_reactions(to_delete, remove_orphans=True)
    
    # Move periplasmic reactions to cytosol
    peri_rxns = set()
    peri_metabs = model.metabolites.query(attribute="compartment", search_function="p")
    peri_cyt_metab_map = {}
    for peri_metab in peri_metabs:
        for rxn in peri_metab.reactions:
            peri_rxns.add(rxn)
        cyt_metab = convert_cmpts(model, peri_metab, from_cmpt='p', to_cmpt='c')
        peri_cyt_metab_map[peri_metab] = cyt_metab
        
    for peri_rxn in peri_rxns:
        peri_rxn.compartments.clear()
        peri_rxn.compartments.update(set('c'))
        peri_rxn_metabs = peri_rxn.metabolites
        coeffs = peri_rxn.get_coefficients(peri_rxn_metabs)
        peri_rxn.subtract_metabolites(dict(zip(peri_rxn_metabs, coeffs)))
        peri_rxn.add_metabolites(dict(zip(map(lambda x: peri_cyt_metab_map[x], peri_rxn_metabs), coeffs)))
            
    model.remove_metabolites(metabolite_list=peri_metabs)"""
    
    # Add exchange reactions for all metabolites
    # Exchange‐reactions for non‐substrate metabolites have lb of 0 and ub of 1000 by default
    #non_sub = set(model.metabolites) - set(map(lambda x: model.reactions.get_by_id(x).metabolites.popitem()[0], model.medium.keys()))
    all_metab_ex = gen_metab_dict(model, model.metabolites, cnvt=False)
    # Exchange reactions for substrates get negative bounds to allow them to enter system
    # Normal medium substrates maintain their lower bounds (effectively unlimited for water, protons -- TODO: check assumption)
    #media = {}
    #for rxn_nm in model.medium:
    #    rxn = model.reactions.get_by_id(rxn_nm)
    #    media[rxn.metabolites.popitem()[0]] = rxn.lower_bound
    # TODO: ensure no duplicates
    #crucial = {model.metabolites.h2o_e: -1000, model.metabolites.o2_e: -1000}
    #all_metab_ex.update(crucial)

    add_ex(model, all_metab_ex)
    model.remove_reactions(['EX_glc_e', 'EX_pi_e', 'EX_mg2_e', 'EX_k_e', 'EX_mg2_e', 'EX_nh4_e'])

    # As objective function, we selected the exchange reaction which corresponds to the target metabolite 
    # for which a pathway should be determined.   
    return model

model_cf = ecoli_to_cf(model)

In [ ]:
s = 0
for rxn_nm in model_cf.medium:
    rxn = model_cf.reactions.get_by_id(rxn_nm)
    if rxn.lower_bound < 0:
        s += 1
print s

In [ ]:
# Based on MetaboTools
# flux = metConc/(cellConc*cellWeight*t*1000);
# cellConc: 10 mg/ml
# cellWeight: 500 * (10 ** -11)
# t = 24
# t: Experiment duration
def conc_to_flux(metab_conc, t=24):
    # Taken from MetaboTools, units are gDW/cell
    #cell_weight = 500 * (10 ** -11)
    # We have 10 mg/ml
    # need cells / ml
    cell_conc = 10 * (1/ 1000.0) #* (1 / cell_weight)
    flux = metab_conc / (cell_conc * t * 1000.0)
    return flux

In [ ]:
def build_medium(mod):
    model = mod.copy()
    # 20 flux is 20 mol/min/gDW
    # in mM
    aa_conc = -1 * conc_to_flux(2)
    trna_conc = -1 * conc_to_flux(0.17)
    metab_conc = map(lambda x: conc_to_flux(x) * -1, [1.2, 0.85, 0.85, 0.85, 0.33, 0.27, 1.50, 1.00, 130, 10, 12, 0.33])
    print metab_conc
    # TODO: map from conc to exchange fluxes
    
    import Bio.PDB.Polypeptide
    aas = map(lambda x: x.lower(), Bio.PDB.Polypeptide.aa3)
    aa_concs = {}
    for aa in aas:
        l_aa, d_aa = None, None
        try:
            l_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '__L_c'), from_cmpt='c', to_cmpt='e')
            d_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '__D_c'), from_cmpt='c', to_cmpt='e')
        except KeyError:
            if aa == 'gly':
                l_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '_c'), from_cmpt='c', to_cmpt='e')
        if d_aa:
            aa_concs[l_aa] = aa_conc / 2.0
            aa_concs[d_aa] = aa_conc / 2.0
        else:
            aa_concs[l_aa] = aa_conc
            
    trna_concs = {convert_cmpts(model, trna_metab, from_cmpt='c', to_cmpt='e') : trna_conc for trna_metab in model.metabolites.query('trna')}
    
    metab_nms = ['atp', 'gtp', 'utp', 'ctp', 'nad', 'coa', 'spmd', 'ptrc', 'k', 'nh4', 'mg2', 'pep']
    metab_concs = gen_metab_dict(model, metab_nms, metab_conc)
        
    # Not included: RNAP & Folinic Acid & cell extract
    base_medium = {}
    base_medium.update(add_ex(model, metab_concs))
    base_medium.update(add_ex(model, aa_concs))
    base_medium.update(add_ex(model, trna_concs))
    
    #model.medium = base_medium
    return model
model_cf_med = build_medium(model_cf)

In [ ]:
opt(model_cf_med)

In [ ]:
alc_dehydr = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='c')
model.add_reaction(alc_dehydr)
alc_dehydr.add_metabolites({'btcoa_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, 'btal_c': 1})

butanol = cobra.Metabolite(id='btol_c', name='1-butanol', compartment='c', charge=0, formula='C4H9OH')
but_synth = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='c')
model.add_reaction(but_synth)
but_synth.add_metabolites({'btal_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, butanol: 1})

In [ ]:
if (not 'DM_btol_c' in mt.reactions):
    mt.add_boundary(metabolite=mt.metabolites.btol_c, type='demand')
mt.objective = mt.reactions.DM_btol_c
mt.optimize()
mt.summary()

In [ ]:
mt.summary(fva=0.95)

In [ ]:
for r in mt.reactions:
    if r.flux > 0.01:
        print r.flux, r.compartments

In [ ]:
df = pd.read_csv('./code/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

In [ ]:
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc__D', 'pi', 'nad', 'atp', 'coa']
for i, row in df.iterrows():
    if i != 4:
        continue
    print row
    with model_cf_med as mod:
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: -1 * conc_to_flux(x), row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=True)
        if (not 'DM_btol_c' in mod.reactions):
            mod.add_boundary(metabolite=mod.metabolites.btol_c, type='demand')
        mod.objective = mod.reactions.DM_btol_c
        #mod.objective = {model_cf_med.reactions.Ec_biomass_iJO1366_core_53p95M: 1, model_cf_med.reactions.DM_btol_c: 1}
        #print mod.medium
        try:
            #sol = gapfill(mod, model, demand_reactions=False)
            print sol[0]
        except:
            print 'Fail'
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        #print mod.summary()
        #print mod.medium
        prod_env = production_envelope(mod, 'DM_btol_c')
        #prod_env = production_envelope(mod, map(lambda x: "EX_{0}_e".format(x), addl_reagent_nms))
        prod_env.head()
        if i >= 0:
            break
#mod.medium

In [ ]:
model.exchanges

In [ ]:
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nad', 'atp', 'coa']
with model_med as mod:
    for i, row in df.iterrows():
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: (x * 100) + 0.001, row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=True)
        #print mod.medium
        try:
            sol = gapfill(mod, model, demand_reactions=False)
        except:
            print 'Fail'
            pass
        print sol[0]
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        mod.repair()
        if i >= 5:
            break

In [ ]:
model_med.add_reactions(reaction_list=[model.reactions.EX_pi_e, model.reactions.EX_pyr_e])

In [ ]:
model_med.remove_reactions([model.reactions.EX_pi_e, model.reactions.EX_pyr_e])

In [ ]:
from cobra.flux_analysis import gapfill
sol = gapfill(model_med, model, demand_reactions=False)
model_med.add_reactions(reaction_list=sol[0])
sol[0]

In [ ]:
if not ('DM_btol_c' in model_med.reactions):
    model_med.add_boundary(model_med.metabolites.btol_c, type='demand')
model_med.objective = model_med.reactions.DM_btol_c
obj = model_med.optimize()
obj.objective_value

In [ ]:
model_med.summary()

In [ ]:
df = pd.read_csv('./code/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

In [ ]:
df.values[0][4:].tolist()

In [ ]:
opt(model)

In [ ]:
model_e.reactions.EX_glc_e

In [ ]:
model_e.medium

In [ ]:
from cobra.flux_analysis.deletion import single_reaction_deletion
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nad', 'atp', 'coa']
with model_e as mod:
    for i, row in df.iterrows():
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: (x * 100) + 0.001, row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=False)
        print mod.medium
        try:
            pass
            #sol = gapfill(mod, model, demand_reactions=False)
        except:
            print 'Fail'
            pass
        #print sol[0]
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        if i >= 5:
            break

In [ ]:
delete_df[delete_df['growth'] < 1.790350]

In [ ]:
biomass = model_e.reactions.get_by_id('Ec_biomass_iJO1366_core_53p95M')

In [ ]:
biomass.reactants

In [ ]:
biomass.products

In [ ]:
biomass

In [ ]:
import escher
import escher.urls

In [ ]:
escher.list_available_models()

In [ ]:
print escher.list_available_maps()
d = escher.urls.root_directory
print('Escher directory: %s' % d)

In [ ]:
json_repr = cobra.io.to_json(model_e)

In [ ]:
b = escher.Builder(model_json=json_repr, reaction_data=sol.fluxes.to_dict())
#b = escher.Builder(map_name='e_coli_core.Core metabolism', reaction_data=sol.fluxes.to_dict())
b.display_in_notebook()

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=solution.x_dict,
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook(js_source='local')

In [ ]:
import Bio.KEGG as bk
from Bio.KEGG import REST
path = REST.kegg_find('PATHWAY', 'n-butanol')
path.readlines()

In [ ]:
re.match()

In [ ]:
rx = re.compile(r'^(?!.*(transport)).*$', re.IGNORECASE)

In [ ]:
rx = re.compile(r'^(?!.*(transport|exchange|flippase|sink)).*$', re.IGNORECASE)
non_trans = eco.reactions.query(attribute='name', search_function=rx)
len(non_trans)

In [ ]:
i = 0
t = set()
for react in eco.reactions:
    if (react.name.find('transport') == -1 and react.subsystem.find('Transport') == -1 and react.name.find('exchange') == -1 and react.name.find('flippase') == -1  and react.name.find('Sink') == -1):
        i += 1
        t.add(react)
i

In [ ]:
from cobra import Model
import copy
eco = model_e.copy()
new_model = Model('new_model')

print("Adding compartment 'a' for 'all'...")
new_model.compartments['a'] = 'all'

#Adding all metabolites from existing model removing duplications
print("Adding all metabolites from existing model removing duplications...")
for x in eco.metabolites:
    dup = False
    for y in new_model.metabolites:
        if x.id[:-2] == y.id:
            dup = True
            break
    if dup == False:
        met = copy.deepcopy(x)
        met.id = met.id[:-2]+'_a'
        met.compartment = 'a'
        new_model.add_metabolites({met})

#Adding all reactions excluding transport, exchange, flippase, sink
print("Adding all reactions excluding transport, exchange, flippase, sink...")
rx = re.compile(r'^(?!.*(transport|exchange|flippase|sink)).*$', re.IGNORECASE)
non_trans = eco.reactions.query(attribute='name', search_function=rx)
for react in non_trans:
    for x in react.reactants:
        n = x.id[:-2]+'_a'
        if not new_model.metabolites.get_by_id(n):
            x.id = x.id[:-2]+'_a'
    for x in react.products:
        n = x.id[:-2]+'_a'
        if not new_model.metabolites.get_by_id(n):
            x.id = x.id[:-2]+'_a'
    new_model.add_reaction(react)
print("Number of reactions in new_model",len(new_model.reactions))

#removing all reactions that are dupliates
print("Removing all duplicate reactions...")
for react in new_model.reactions:
    for y in new_model.reactions:
        if react.reaction == y.reaction and react.name != y.name:
            y.delete()
print("Number of reactions after deleting duplicates: ",len(new_model.reactions))


# adding necessary metabolites and reactions for biosensor project
print("Adding necessary metabolites and reactions for biosensor project...")

In [ ]:
alc_dehydr_a = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='a')
new_model.add_reaction(alc_dehydr_a)
alc_dehydr_a.add_metabolites({'btcoa_a': -1, 'h_a': -1, 'nadh_a': -1, 'nad_a': 1, 'btal_a': 1})

butanol_a = cobra.Metabolite(id='btol_a', name='1-butanol', compartment='a', charge=0, formula='C4H9OH')
but_synth_a = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='a')
new_model.add_reaction(but_synth_a)
but_synth_a.add_metabolites({'btal_a': -1, 'h_a': -1, 'nadh_a': -1, 'nad_a': 1, butanol: 1})

new_model.add_boundary(new_model.metabolites.btol_a, type='demand')

In [ ]:
new_model.metabolites.bto

In [ ]:
new_model.objective = new_model.reaction.DM_btol_a
new_model.optimize()
new_model.summary()

In [ ]:
m = cobra.io.load_json_model(filename='./code/varner.json')

In [ ]:
m.metabolites.M_h_c_exchange_reverse

In [ ]:
m.reactions.M_h_c_exchange_reverse

In [ ]:
m.objective = m.reactions.but_export

In [ ]:
m.objective.expression

In [ ]:
help(m.problem.Model)

In [ ]:
m.problem.glp_write_lp(m.problem.Model, None, './code/py_prob.cplx')

In [ ]:
import swiglpk
prob = swiglpk.glp_create_prob()

In [ ]:
swiglpk.glp_read_lp(prob, None, './code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/prob.cplx')

In [ ]:
prob.

In [ ]:
import optlang
from optlang import glpk_interface
optlang_prob = optlang.glpk_interface.glp_create_prob()

In [ ]:
optlang.glpk_interface.glp_read_lp(optlang_prob, None, './code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/prob.cplx')

In [ ]:
opt_mod = glpk_interface.Model(problem=optlang_prob)

In [ ]:
opt_mod.optimize()

In [ ]:
opt_mod.primal_values

In [ ]:
import json

In [ ]:
js = json.loads(cobra.io.to_json(model))

In [ ]:
js.keys()

In [ ]:
js['version']

In [ ]:
js['metabolites'][0]

In [ ]:
js['reactions']

In [ ]:
js['compartments']

In [ ]:
model.reactions.CDAPPA160

In [ ]:
mod_simp = cobra.test.create_test_model('textbook')

In [ ]:
mod_simp